In [59]:
'''Created By: Jake Fredrich'''
'''Date: 11/12/2019'''
'''Purpose: Store a table from a wikipedia page to a dataframe to be used in a future lab.
   The prompt is from Applied Data Science: Capstone by Coursera'''
'''RESOURCES:
    >Coursera
    >Stackoverflow.com
    >Kaggle.com
    >https://github.com'''

'RESOURCES:\n    >Coursera\n    >Stackoverflow.com\n    >Kaggle.com\n    >https://github.com'

# Import necessary Packages

In [60]:
'''Used for ssl creation'''
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

!conda install -c conda-forge geocoder --yes

print('complete')

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.1.23-0, anaconda::openssl-1.1.1b-he774522_1
  - anaconda::openssl-1.1.1b-he774522_1, defaults::ca-certificates-2019.1.23-0
  - anaconda::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-he774522_1
  - defaults::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-he774522_1done

# All requested packages already installed.

complete


In [61]:
'''IMPORTS REQUIRED'''
from bs4 import BeautifulSoup # used for pulling data out of HTML and XML files
import json # library to handle JSON files
import numpy as np
import pandas as pd
import requests # library to handle API requests
from pandas.io.json import json_normalize #used to transform the JSON file into a pandas Dataframe

import matplotlib.cm as cm # for plotting modules
import matplotlib.colors as colors # for plotting modules
import urllib # for plotting modules

from sklearn.cluster import KMeans # import k-means from clustering
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # Used to convert addresses into latitude and longitudes
#!conda install -c conda-forge folium=0.5.0 --yes

import folium # Used to render maps
print("Packages imported")

Packages imported


# Parse Wikipedia Page using Beautiful Soup

In [62]:
'''SCRAPE Wikipedia PAGE FOR TORONTO'''

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # Save URL
response = urllib.request.urlopen(url)
type(response)
html = response.read() # Scraped HTML
print(html)
soup = BeautifulSoup(html, 'html.parser') # Create BeautifulSoup object

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XcXJQwpAAEIAACRL9AsAAAAW","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in 

In [63]:
''' LOCATE TABLE & EXPLORE PULLING'''
# Located the table by inspecting elements. table class="wikitable"

pcodeList = []
for row in soup.select('table.wikitable tr'): # Iterate through the table object
    cells = row.findAll('td') # Find all of the 'td''s
    if len(cells) > 0: # If 'td''s exist, then
        post_codes = cells[0].text.strip() # set post_codes equal to the first segment
        borough = cells[1].text.strip() # set borough equal to the second segment
        neighborhood = cells[2].text.strip() # set neighborhood equal to the third segment
        
        if (borough != "Not assigned" and neighborhood != 'Not assigned'): # Append if both exist
            pcodeList.append([post_codes, borough, neighborhood])
        elif(borough != 'Not assigned'): 
            pcodeList.append([post_codes, borough, borough]) # Append if Borough exists
           
   
T_df = pd.DataFrame(pcodeList, columns=['PCodes', 'Borough', 'Neighborhood']) # Create dataframe using list
T_df = T_df.sort_values(by=['PCodes']) # Sort
print(T_df.head(15))
T_df.shape


   PCodes      Borough          Neighborhood
8     M1B  Scarborough                 Rouge
9     M1B  Scarborough               Malvern
23    M1C  Scarborough            Port Union
22    M1C  Scarborough            Rouge Hill
21    M1C  Scarborough        Highland Creek
33    M1E  Scarborough             Guildwood
34    M1E  Scarborough           Morningside
35    M1E  Scarborough             West Hill
39    M1G  Scarborough                Woburn
43    M1H  Scarborough             Cedarbrae
54    M1J  Scarborough   Scarborough Village
66    M1K  Scarborough  East Birchmount Park
67    M1K  Scarborough               Ionview
68    M1K  Scarborough          Kennedy Park
80    M1L  Scarborough           Golden Mile


(211, 3)

In [64]:
''' TAKE PRIOR CODE AND FORMAT NEIGHBORHOODS WITH SAME POSTAL CODES'''
# Located the table by inspecting elements. table class="wikitable"
pcodeList = []
prev_post_code = "" # store previous postal code
all_neighbourhoods = ""
prev_borough = ""

for row in soup.select('table.wikitable tr'): # Iterate through the table object
    cells = row.findAll('td') # Find all of the 'td''s
    if len(cells) > 0: # If 'td''s exist, then
        post_code = cells[0].text.strip() # set post_codes equal to the first segment
        borough = cells[1].text.strip() # set borough equal to the second segment
        neighborhood = cells[2].text.strip() # set neighborhood equal to the third segment
        
        if ((borough == "Not assigned" and neighborhood == 'Not assigned') or borough == "blank"): # Append if both exist
            #Skip
            Skip = "" # Has no value, ignore
        else: # If not skipped
            if neighborhood == 'Not assigned': # set Neighborhood = Borough when not assigned
                neighborhood = borough 
            
            if prev_post_code == post_code: # append neighborhood to all_neighborhoods when duplicates exist
                #print(post_code)
                all_neighbourhoods = all_neighbourhoods + ", " + neighborhood   
            else:
                if prev_post_code != "": # append when all grouped neighborhoods complete
                    pcodeList.append([prev_post_code, prev_borough, all_neighbourhoods])
                prev_post_code = post_code # set lsat post code = to current
                prev_borough = borough # set lsat borough = to current
                all_neighbourhoods = neighborhood #start a new neighborhoods string
                
pcodeList.append([prev_post_code, prev_borough, all_neighbourhoods]) # append last row
    

T_df = pd.DataFrame(pcodeList, columns=['Postal Code', 'Borough', 'Neighborhood']) # Create dataframe using list
T_df = T_df.sort_values(by=['Postal Code']) # Sort
T_df.head()

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,"Rouge, Malvern"
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [65]:
T_df.shape # print shape of T_df

(103, 3)

# Create Latitude and Longitude Fields to DataFrame

In [66]:
''' FAILED TO GET GEOCODER TO WORK PROPERLY '''
#import geocoder # import geocoder
#postal_code = T_df['PCodes'].to_list() #store postal_codes to list
#print(postal_code[1])
# initialize your variable to None
#lat_lng_coords = None


# loop until you get the coordinates
#while(lat_lng_coords is None):
    #g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
    #lat_lng_coords = g.latlng
    #print(type(g))

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print("hi")


' FAILED TO GET GEOCODER TO WORK PROPERLY '

In [67]:
''' READ IN GEOSPACIAL_COORDINATES CSV '''

File_Name = "Geospacial_Coordinates.csv"
geo_df = pd.read_csv(File_Name) # Read File
geo_df.head()

T_df = pd.merge(T_df, geo_df, on="Postal Code")
T_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Clustering

##### Check the size of T_df

In [68]:
print('The dataframe has {} boroughs.'.format(
    len(T_df['Borough'].unique())))

The dataframe has 11 boroughs.


##### Get the latitude/longitude of Toronto

In [69]:
address = "Toronto, ON" # set Address

geolocator = Nominatim()
location = geolocator.geocode(address) # store location found using address
latitude = location.latitude # store latitude of Toronto
longitude = location.longitude # store longitude of Toronto
print('The coordinates of Tornto are {}, {}.'.format(latitude, longitude))

C:\Users\JAKEFREDRICH\Anaconda3\envs\BaseClone\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The coordinates of Tornto are 43.653963, -79.387207.


##### Create map of Toronto's neighborhoods

In [70]:
T_map = folium.Map(location=[latitude, longitude], zoom_start=10) # create map of Toronto using long n lat

#add markers
for lat, lng, borough, neighborhood in zip(T_df['Latitude'], T_df['Longitude'], T_df['Borough'],
                                          T_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(T_map)
        
T_map

##### Filter Out All Boroughs that Aren't "Toronto"

In [71]:
T2_df = T_df[T_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
T2_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [72]:
# set the Latitude and longitude to the mean of both lat & long
latitude = T2_df['Latitude'].mean()
longitude = T2_df['Longitude'].mean()
latitude


43.66726218421052

In [73]:
#Remap with new items
T2_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(T2_df['Latitude'], T2_df['Longitude'], T2_df['Borough'], T2_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(T2_map)

T2_map

In [74]:
'''SET UP FOURSQUARE CREDENTIALS'''
CLIENT_ID = "E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW"
CLIENT_SECRET = "XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW"
VERSION = '20180604'
LIMIT = 30

print('Your Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your Credentials:
CLIENT_ID: E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW
CLIENT_SECRET:XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW


##### Explore the First Neighborhood

In [75]:
neighborhood = T2_df.iat[1,2]
print("Lat & Long values of {} are {}, {}.".format(neighborhood, latitude, longitude))


Lat & Long values of The Danforth West, Riverdale are 43.66726218421052, -79.38988323421053.


##### Get top venues

In [76]:
LIMIT=100
radius = 500
# type your answer here
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,CLIENT_SECRET,VERSION,
        latitude,longitude,
        radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW&client_secret=XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW&v=20180604&ll=43.66726218421052,-79.38988323421053&radius=500&limit=100


In [77]:
results = requests.get(url).json() # Json file to be converted
#results

In [78]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))



                   name      categories        lat        lng
0                Indigo       Bookstore  43.669065 -79.389057
1      Bay Street Video     Video Store  43.668890 -79.389247
2                   COS  Clothing Store  43.669516 -79.390390
3                Pi Co.     Pizza Place  43.670107 -79.389852
4  Harry Rosen Menswear     Men's Store  43.669661 -79.390585
100 venues were returned by Foursquare.


##### Function to find nearest venues using Lab Function

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
        

In [80]:
T_venues = getNearbyVenues(names=T2_df['Neighborhood'], latitudes=T2_df['Latitude'], longitudes=T2_df['Longitude'])
print(T_venues.head())
print(T_venues.shape)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [81]:
T_venues.groupby('Neighborhood').count() # get count of venues per neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",49,49,49,49,49,49
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,16,16,16,16,16,16


##### Analyze neighborhoods

In [82]:
# one hot encoding
T_onehot = pd.get_dummies(T_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column to the dtatframe
T_onehot['Neighborhood'] = T_venues['Neighborhood']

# move neighborhood column to the first column
#fixed_columns = [T_onehot.columns[-1]] + list(T_onehot.columns[:-1])
#fixed_columns = [T_onehot.columns[T_onehot.columns.get_loc("Neighborhood")]] + list(T_onehot.columns).remove("Neighborhood")
nTemp = T_onehot['Neighborhood']
T_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
T_onehot.insert(0, 'Neighborhood', nTemp)



#print(fixed_columns)


#T_onehot = T_onehot[fixed_columns]

#T_onehot.set_index('Neighborhood', inplace = True)


T_onehot.head()
print(T_onehot.shape)

(1691, 228)


In [83]:
''' GROUP by frequency '''
T_grouped = T_onehot.groupby('Neighborhood').mean().reset_index()
T_grouped.head()
#T_grouped.shape



,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.03,0.0,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.00,0.0,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.066667,0.066667,0.133333,0.2,0.066667,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000


##### Print each neighborhood with their most popular venues

In [84]:
num_top_venues = 10

for hood in T_grouped['Neighborhood']:
    print("----"+str(hood)+"----")
    temp = T_grouped[T_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                  Bar  0.04
3           Steakhouse  0.04
4                Hotel  0.03
5       Cosmetics Shop  0.03
6     Sushi Restaurant  0.03
7  American Restaurant  0.03
8           Restaurant  0.03
9     Asian Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.04
2              Bakery  0.04
3  Seafood Restaurant  0.04
4        Cocktail Bar  0.04
5         Cheese Shop  0.04
6          Steakhouse  0.04
7            Beer Bar  0.04
8      Farmers Market  0.04
9         Art Gallery  0.02


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.09
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3                     Bar  0.04
4              Restaurant  0.04
5    Caribbean Restaurant  0.04
6             Music Venue  0.04
7      

                             venue  freq
0                              Gym   1.0
1                Afghan Restaurant   0.0
2                        Nightclub   0.0
3         Mediterranean Restaurant   0.0
4                      Men's Store   0.0
5               Mexican Restaurant   0.0
6        Middle Eastern Restaurant   0.0
7               Miscellaneous Shop   0.0
8       Modern European Restaurant   0.0
9  Molecular Gastronomy Restaurant   0.0


----North Toronto West----
                 venue  freq
0       Clothing Store  0.14
1          Coffee Shop  0.10
2  Sporting Goods Shop  0.10
3          Yoga Studio  0.05
4           Bagel Shop  0.05
5         Dessert Shop  0.05
6                Diner  0.05
7                 Park  0.05
8  Rental Car Location  0.05
9           Restaurant  0.05


----Parkdale, Roncesvalles----
                         venue  freq
0                    Gift Shop  0.13
1                  Coffee Shop  0.13
2               Breakfast Spot  0.07
3           Italian 

In [85]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [86]:
num_top_venues = 10 # set number of venues we'd like to see

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues - ind is indicator 
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = T_grouped['Neighborhood']

for ind in np.arange(T_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(T_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.set_index('Neighborhood', inplace = True)
neighborhoods_venues_sorted.dtypes

Neighborhood              object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object

## CLUSTER
##### K-means algorithym 

In [111]:
# Cluster num
kclusters = 7

# Remove Neighborhood
T_grouped_clustering =  T_grouped.drop('Neighborhood', 1) 

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(T_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

#neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_venues_sorted['Neighborhood'].str[1:]
#neighborhoods_venues_sorted.head()

#neighborhoods_venues_sorted['Latitude'] = pd.to_numeric(df["2nd"])

print(type(neighborhoods_venues_sorted['Neighborhood']))
print(neighborhoods_venues_sorted.dtypes)

<class 'pandas.core.series.Series'>
Neighborhood              object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object


##### Merged Data with  more info

In [112]:
T_merged = T2_df
#T_merged.set_index("Neighborhood", inplace = True)

# add clustering labels
T_merged['Cluster Labels'] = kmeans.labels_

#print(T_merged.dtypes)
#T_merged.head()
# merge T_grouped with toronto_data to add latitude/longitude for each neighborhood
T_merged = T_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
T_merged.head(10) # check the last columns!



,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Burrito Place,Ice Cream Shop,Food & Drink Shop,Fish & Chips Shop,Pub,Movie Theater,Italian Restaurant,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Diner,Fish Market,Middle Eastern Restaurant,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Breakfast Spot,Park,Clothing Store,Food & Drink Shop,Hotel,Sandwich Place,Gym,Event Space,Ethiopian Restaurant,Dog Run
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Coffee Shop,Sushi Restaurant,Gym,Park,Indian Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Gym,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Pub,Coffee Shop,Pizza Place,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


##### Visualize

In [113]:
# Create Map
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=11)

# Set colors for clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers
markers_colors = []
for lat, lon, poi, cluster in zip(T_merged['Latitude'], T_merged['Longitude'],
                                 T_merged['Neighborhood'], T_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=.7).add_to(map_clusters)
map_clusters

##### Examine


In [114]:
# look at first Cluster
T_merged.loc[T_merged['Cluster Labels'] == 0, T_merged.columns[[2] +
        list(range(5, T_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Health Food Store,Pub,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant,Restaurant
2,"The Beaches West, India Bazaar",0,Park,Sandwich Place,Burrito Place,Ice Cream Shop,Food & Drink Shop,Fish & Chips Shop,Pub,Movie Theater,Italian Restaurant,Fast Food Restaurant
3,Studio District,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Diner,Fish Market,Middle Eastern Restaurant,Clothing Store
4,Lawrence Park,0,Park,Bus Line,Swim School,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,Davisville North,0,Breakfast Spot,Park,Clothing Store,Food & Drink Shop,Hotel,Sandwich Place,Gym,Event Space,Ethiopian Restaurant,Dog Run
6,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
7,Davisville,0,Pizza Place,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Coffee Shop,Sushi Restaurant,Gym,Park,Indian Restaurant
8,"Moore Park, Summerhill East",0,Gym,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Pub,Coffee Shop,Pizza Place,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


In [115]:
# look at second Cluster
T_merged.loc[T_merged['Cluster Labels'] == 1, T_merged.columns[[2] +
        list(range(5, T_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,"CN Tower, Bathurst Quay, Island airport, Harbo...",1,Airport Service,Airport Lounge,Harbor / Marina,Coffee Shop,Airport,Airport Food Court,Airport Terminal,Bar,Sculpture Garden,Plane


In [116]:
# look at third Cluster
T_merged.loc[T_merged['Cluster Labels'] == 2, T_merged.columns[[2] +
        list(range(5, T_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"The Annex, North Midtown, Yorkville",2,Sandwich Place,Café,Coffee Shop,Pizza Place,BBQ Joint,Pet Store,Pharmacy,Cosmetics Shop,Pub,Burger Joint


In [117]:
# look at fourth Cluster
T_merged.loc[T_merged['Cluster Labels'] == 3, T_merged.columns[[2] +
        list(range(5, T_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Stn A PO Boxes 25 The Esplanade,3,Coffee Shop,Restaurant,Café,Seafood Restaurant,Italian Restaurant,Hotel,Beer Bar,Gym,Bakery,Breakfast Spot


In [118]:
# look at fifth Cluster
T_merged.loc[T_merged['Cluster Labels'] == 4, T_merged.columns[[2] +
        list(range(5, T_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,"Parkdale, Roncesvalles",4,Gift Shop,Coffee Shop,Movie Theater,Restaurant,Bookstore,Eastern European Restaurant,Italian Restaurant,Bar,Bank,Dog Run
